In [ ]:
import pandas as pd
import requests
import certifi
import tweepy
import time
from datetime import datetime, timedelta

# python3 -m pip install pandas requests tweepy
# python3 -m pip install --upgrade pip
# pip install --upgrade pip pandas requests tweepy certifi urllib3

# python3 -c "import certifi; print(certifi.where())"
# export SSL_CERT_FILE=$(python3 -c "import certifi; print(certifi.where())")
# python3 ./chainsbot.py

# ... code for get_chain_control_status ...
def get_chain_control_status(route='US-50'):
    url = "https://cwwp2.dot.ca.gov/data/d3/cc/ccStatusD03.csv"
    try:
        response = requests.get(url)
        response.raise_for_status()
        df = pd.read_csv(url)
        filtered_df = df[df['route'] == route]
        eastbound_df = filtered_df[filtered_df['direction'] == 'East']
        westbound_df = filtered_df[filtered_df['direction'] == 'West']

        def get_status_info(direction_df, status_code, min_or_max):
            status_df = direction_df[direction_df['status'] == status_code]
            if not status_df.empty:
                if min_or_max == 'min':
                    postmile = status_df['postmile'].min()
                else:
                    postmile = status_df['postmile'].max()
                location = status_df[status_df['postmile'] == postmile]['locationName'].iloc[0]
                return postmile, location
            return None, None

        eastbound_r2_min_postmile, eastbound_r2_location = get_status_info(eastbound_df, 'R-2', 'min')
        westbound_r2_max_postmile, westbound_r2_location = get_status_info(westbound_df, 'R-2', 'max')
        eastbound_ht_min_postmile, eastbound_ht_location = get_status_info(eastbound_df, 'HT', 'min')
        westbound_ht_max_postmile, westbound_ht_location = get_status_info(westbound_df, 'HT', 'max')
        eastbound_rc_min_postmile, eastbound_rc_location = get_status_info(eastbound_df, 'RC', 'min')
        westbound_rc_max_postmile, westbound_rc_location = get_status_info(westbound_df, 'RC', 'max')

        status_message = ""
        if not any([eastbound_r2_min_postmile, westbound_r2_max_postmile, eastbound_ht_min_postmile, westbound_ht_max_postmile, eastbound_rc_min_postmile, westbound_rc_max_postmile]):
            status_message = "🚙 OPEN: No chain controls"
        else:
            if eastbound_rc_min_postmile: status_message += f"⛔️ CLOSED at {eastbound_rc_location}\n"
            if westbound_rc_max_postmile: status_message += f"⛔️ CLOSED at {westbound_rc_location}\n"
            if eastbound_ht_min_postmile: status_message += f"⚠️ HELD at {eastbound_ht_location}\n"
            if westbound_ht_max_postmile: status_message += f"⚠️ HELD at {westbound_ht_location}\n"
            if eastbound_r2_min_postmile: status_message += f"⛓️ CHAINS EAST at {eastbound_r2_location}\n"
            if westbound_r2_max_postmile: status_message += f"⛓️ CHAINS WEST at {westbound_r2_location}\n"

        return status_message
    except requests.exceptions.RequestException as e:
        return f"Error downloading the file: {e}"
    except pd.errors.ParserError as e:
        return f"Error parsing the CSV file: {e}"
    except KeyError as e:
        return f"Error: Column '{e}' not found in the DataFrame"
    except IndexError as e:
        return f"Error: {e}. Check if any filters resulted in empty DataFrames"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

# actual Twitter API credentials
bearer_token = "some token"
consumer_key = "some key"
consumer_secret = "some secret"
access_token = "another token"
access_token_secret = "another secret"

client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret
)

previous_status = ""
posts_today = 0
today = datetime.now().date()

while True:
    try:
        current_status = get_chain_control_status()
        if current_status != previous_status:
            now = datetime.now()
            if now.date() > today:
                today = now.date()
                posts_today = 0  # Reset daily post count

            if posts_today < 10:
                print("Status changed. Posting to Twitter...")
                print(current_status)
                try:
                    response = client.create_tweet(text=current_status)
                    print(f"Tweet posted successfully with ID: {response.data['id']}")
                    previous_status = current_status
                    posts_today += 1
                except tweepy.errors.TweepyException as e:
                    print(f"Error posting tweet: {e}")
            else:
                print("Maximum posts for today reached.")
        else:
            print("No change in status.")
        time.sleep(300)  # Wait for 5 minutes (300 seconds)
    except Exception as e:
        print(f"An unexpected error occurred in the main loop: {e}")
        time.sleep(300) #Wait 5 minutes before retrying

